In [1]:
import pickle
from datetime import datetime
from os import mkdir
from os.path import join, exists

import numpy as np
import pandas as pd
import scipy as sp

from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [2]:
train_df = pd.read_pickle('../dataset/train.pkl')

In [3]:
test_df = pd.read_pickle('../dataset/test.pkl')

## 目前占比最高的依次为

- hotel_roomid_rank,0.036305468529462814
- basicroomid__room_30days_ordnumratio_max,0.02795938216149807
- order_price_dif_hotel_rt,0.026568368077278137
- order_price_ave_star_dif,0.019752399995923042
- order_return_dx,0.019474197179079056
- order_price_dif_rt,0.018917791545391083
- roomservice_8,0.018083183094859123

## 添加 room_30days_ordnumratio  计数特征（一大半为NAN）

## 添加 basic_30days_ordnumratio 计数特征

In [7]:
train_df.loc[train_df.basic_30days_ordnumratio.isnull()].info(null_counts=True, verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124750 entries, 193 to 1094737
Data columns (total 151 columns):
orderid                               124750 non-null int32
uid                                   124750 non-null int32
orderdate                             124750 non-null datetime64[ns]
hotelid                               124750 non-null int32
basicroomid                           124750 non-null int32
roomid                                124750 non-null int32
orderlabel                            124750 non-null int8
star                                  124750 non-null int8
rank                                  124750 non-null int16
returnvalue                           124750 non-null float32
price_deduct                          124750 non-null float32
basic_minarea                         124750 non-null float16
basic_maxarea                         124750 non-null float16
roomservice_1                         124750 non-null int8
roomservice_2                  

In [4]:
train_df.basic_30days_ordnumratio.isnull().sum()

124750

In [48]:
train_df[['orderlabel', 'room_30days_ordnumratio']].corr()

,orderlabel,room_30days_ordnumratio
orderlabel,1.000000,0.210727
room_30days_ordnumratio,0.210727,1.000000


In [54]:
train_df['room_30days_ordnumratio_isnull'] = 0

In [55]:
train_df.loc[train_df.room_30days_ordnumratio.isnull(), 'room_30days_ordnumratio_isnull'] = 1

In [56]:
train_df[['orderlabel', 'room_30days_ordnumratio', 'room_30days_ordnumratio_isnull']].corr()

,orderlabel,room_30days_ordnumratio,room_30days_ordnumratio_isnull
orderlabel,1.000000,0.210727,-0.103385
room_30days_ordnumratio,0.210727,1.000000,NaN
room_30days_ordnumratio_isnull,-0.103385,NaN,1.000000


In [53]:
train_df.loc[(train_df.room_30days_ordnumratio.isnull())&(train_df.orderdate_lastord.isnull())].shape

(40443, 153)

In [51]:
train_df.room_30days_ordnumratio.isnull().sum()

457994

In [49]:
train_df.room_30days_ordnumratio.describe()

count    636832.000000
mean          0.060913
std           0.098877
min           0.000225
25%           0.007935
50%           0.024139
75%           0.069580
max           1.000000
Name: room_30days_ordnumratio, dtype: float64

### hotel_roomid_rank 订单内每个子房型在物理房型的位置

取值依次为 3，5， 7， 9 ---- 

从3开始起，依次递增

相关性为（-0.08）

当不小心删除该特征时，惊奇的发现roomservice_8变成了第一，所以使用一个交叉特征

In [35]:
train_df[['roomtag_1', 'orderlabel']].corr()

,roomtag_1,orderlabel
roomtag_1,1.000000,0.392016
orderlabel,0.392016,1.000000


In [20]:
train_df[['rank', 'orderlabel']].corr()

,rank,orderlabel
rank,1.000000,-0.080251
orderlabel,-0.080251,1.000000


In [26]:
train_df['rank_roomservice_8'] = (
    train_df['roomservice_8'].astype(str) +
    train_df['rank'].astype(str)).astype('category').cat.codes

In [30]:
train_df[['rank', 'rank_roomservice_8', 'roomservice_8', 'orderlabel']].corr()

,rank,rank_roomservice_8,roomservice_8,orderlabel
rank,1.000000,0.095208,0.125574,-0.080251
rank_roomservice_8,0.095208,1.000000,0.881881,-0.081412
roomservice_8,0.125574,0.881881,1.000000,-0.058262
orderlabel,-0.080251,-0.081412,-0.058262,1.000000


In [ ]:
train_df.price_deduct

In [31]:
train_df['roomid_price_rank'] = train_df.groupby(['orderid', 'basicroomid']).price_deduct.rank()

In [34]:
train_df.roomid_price_rank

0          3.0
1          1.0
2          4.0
3          9.0
4          3.0
5          4.5
6          6.0
7          1.0
8          2.5
9          2.5
10         2.0
11         1.5
12         1.5
13         2.0
14         5.0
15         3.5
16         5.5
17         8.0
18         1.5
19         7.0
20         4.5
21         6.0
22         7.0
23         2.0
24         4.0
25         7.5
26         3.5
27         5.5
28         9.0
29         6.0
          ... 
1094796    7.0
1094797    4.0
1094798    5.0
1094799    1.0
1094800    2.0
1094801    3.0
1094802    6.0
1094803    5.0
1094804    5.0
1094805    6.0
1094806    1.0
1094807    7.0
1094808    6.0
1094809    7.0
1094810    6.0
1094811    7.0
1094812    5.0
1094813    2.0
1094814    3.0
1094815    6.0
1094816    6.0
1094817    1.0
1094818    3.0
1094819    7.0
1094820    5.0
1094821    7.0
1094822    2.0
1094823    5.0
1094824    7.0
1094825    4.0
Name: roomid_price_rank, Length: 1094826, dtype: float64

In [33]:
train_df[['rank', 'rank_roomservice_8', 'roomservice_8', 'roomid_price_rank', 'roomid_price_rank',  'orderlabel']].corr()

,rank,rank_roomservice_8,roomservice_8,roomid_price_rank,roomid_price_rank,orderlabel
rank,1.000000,0.095208,0.125574,0.937218,0.937218,-0.080251
rank_roomservice_8,0.095208,1.000000,0.881881,-0.049965,-0.049965,-0.081412
roomservice_8,0.125574,0.881881,1.000000,-0.061815,-0.061815,-0.058262
roomid_price_rank,0.937218,-0.049965,-0.061815,1.000000,1.000000,-0.078895
roomid_price_rank,0.937218,-0.049965,-0.061815,1.000000,1.000000,-0.078895
orderlabel,-0.080251,-0.081412,-0.058262,-0.078895,-0.078895,1.000000


In [17]:
train_df.loc[train_df.orderid==92120]

,orderid,uid,orderdate,hotelid,basicroomid,roomid,orderlabel,star,rank,returnvalue,...,roomservice_6_lastord,roomservice_8_lastord,roomtag_2_lastord,roomtag_3_lastord,roomtag_4_lastord,roomtag_5_lastord,star_lastord,hotel_minprice_lastord,basic_minprice_lastord,hotel_roomid
1094738,92120,132585,2017-06-10 20:56:24,62135,295960,10654717,0,9,7,200.0,...,2.0,1.0,NaN,NaN,0.0,0.0,5.0,854.0,890.0,95709
1094739,92120,132585,2017-06-10 20:56:24,62135,295960,10654749,0,9,9,200.0,...,2.0,1.0,NaN,NaN,0.0,0.0,5.0,854.0,890.0,95722
1094740,92120,132585,2017-06-10 20:56:24,62135,295961,106548411,0,9,11,200.0,...,2.0,1.0,NaN,NaN,0.0,0.0,5.0,854.0,890.0,95764
1094741,92120,132585,2017-06-10 20:56:24,62135,295972,10655645,0,9,5,200.0,...,2.0,1.0,NaN,NaN,0.0,0.0,5.0,854.0,890.0,96098
1094742,92120,132585,2017-06-10 20:56:24,62135,295961,10654819,1,9,9,200.0,...,2.0,1.0,NaN,NaN,0.0,0.0,5.0,854.0,890.0,95756
1094743,92120,132585,2017-06-10 20:56:24,62135,295966,106552211,0,9,11,200.0,...,2.0,1.0,NaN,NaN,0.0,0.0,5.0,854.0,890.0,95918
1094744,92120,132585,2017-06-10 20:56:24,62135,295967,10655283,0,9,3,200.0,...,2.0,1.0,NaN,NaN,0.0,0.0,5.0,854.0,890.0,95940
1094745,92120,132585,2017-06-10 20:56:24,62135,295964,10655069,0,9,9,200.0,...,2.0,1.0,NaN,NaN,0.0,0.0,5.0,854.0,890.0,95861
1094746,92120,132585,2017-06-10 20:56:24,62135,295967,106552911,0,9,11,200.0,...,2.0,1.0,NaN,NaN,0.0,0.0,5.0,854.0,890.0,95941
1094747,92120,132585,2017-06-10 20:56:24,62135,295971,106556015,0,9,15,200.0,...,2.0,1.0,NaN,NaN,0.0,0.0,5.0,854.0,890.0,96084


In [19]:
train_df.loc[(train_df.orderid==92120)&(train_df.basicroomid==295967)]['rank']

1094744     3
1094746    11
1094749     7
1094781     9
1094808    13
1094809    15
1094813     5
Name: rank, dtype: int16

In [15]:
train_df.loc[(train_df.orderid==94304)&(train_df.basicroomid==25981)]['rank']

11     3
17    17
18     5
19    15
20    11
29    13
30    19
31     7
35     9
Name: rank, dtype: int16